In [9]:
import requests
from astral.sun import sun
from astral import LocationInfo
from datetime import datetime, timedelta, timezone


# 获取公网IP地址
response = requests.get('https://ipinfo.io')
data = response.json()
# 提取经度和纬度
coordinates = data['loc'].split(',')
latitude = coordinates[0]
longitude = coordinates[1]
# 创建一个 LocationInfo 对象，只提供经纬度信息
location = LocationInfo("", "", "", latitude=latitude, longitude=longitude)
# 获取当前日期和时间，并为其添加时区信息
now = datetime.now(timezone.utc)
def sunrise_sunset(time):
    # 创建一个 Sun 对象
    sun_time = sun(location.observer, date=time)
    # 计算日出和日落时间，以及日落前和日出后的一小时
    sunrise = sun_time['sunrise']
    sunset = sun_time['sunset']
    sunrise_minus_one_hour = sunrise - timedelta(hours=1)
    sunset_plus_one_hour = sunset + timedelta(hours=1)
    return sunrise_minus_one_hour, sunset_plus_one_hour
sunrise_now, sunset_now = sunrise_sunset(now)
if sunrise_now < sunset_now:
    if sunrise_now < now < sunset_now:
        print("现在是白天")
    else:
        print("现在是晚上")
else:
    if sunrise_now < now < sunset_now:
        print("现在是晚上")
    else:
        print("现在是白天")

现在是白天
2023-10-15 13:15:28.480962+00:00 2023-10-15 02:11:21.356445+00:00


In [4]:
import requests
import datetime
import math

# 获取当前日期和时间
now = datetime.datetime.now()
# 获取当前日期是一年中的第几天
day_of_year = now.timetuple().tm_yday
# 获取公网IP地址
response = requests.get('https://ipinfo.io')
data = response.json()
# 提取经度和纬度
coordinates = data['loc'].split(',')
latitude = float(coordinates[0])
longitude = float(coordinates[1])
# 计算太阳的高度角
# 使用简化的算法，不考虑大气折射等因素
solar_declination = -23.44 * math.cos(math.radians(360 * (day_of_year + 10) / 365))
hour_angle = 15 * (now.hour - 12) + (now.minute / 4) + (longitude / 15)
solar_altitude = math.degrees(math.asin(
    math.sin(math.radians(latitude)) * math.sin(math.radians(solar_declination)) +
    math.cos(math.radians(latitude)) * math.cos(math.radians(solar_declination)) * math.cos(math.radians(hour_angle)
)))
# 根据太阳的高度角判断是白天还是晚上
if solar_altitude > 0:
    print("现在是白天")
else:
    print("现在是晚上")


现在是白天
